# 05 – Model Training & Evaluation  
**CRISP-DM Phase 4: Modeling** (and Phase 5: Evaluation)  
This notebook fits a regression model, evaluates its performance, and serialises artifacts for deployment.

### Objectives
* One-hot encode features to match model expectations.  
* Split data 80 / 20 (with `random_state=42`).  
* Fit a baseline `LinearRegression` model.  
* Evaluate on test set: R², MAE, RMSE.  
* Serialize artifacts for deployment:  
  - `house_price_model.pkl`  
  - `model_columns.pkl`  
  - *(optional)* `model_metrics.json`  

### Inputs
* `outputs/datasets/collection/HousePricesRecords_clean.csv`  

### Outputs
* `outputs/models/house_price_model.pkl`  
* `outputs/models/model_columns.pkl`  
* *(optional)* `outputs/models/model_metrics.json`  

### Additional Comments  
#### Business Requirements Addressed  
* **BR3**: Produces the trained model for the Sale Price Prediction tab.  

#### Additional Notes  
* Later: upgrade to a pipeline with XGBoost + hyperparameter tuning to boost performance. 

### Import Required Libraries for Modeling & Evaluation  
This cell brings in the modules we’ll need to load data and the trained model, split the dataset, fit our regression algorithm, and compute performance metrics:

- **`os`** for file‐system operations (ensuring output folders exist, constructing paths).  
- **`joblib`** to deserialize the previously saved `house_price_model.pkl` and `model_columns.pkl`.  
- **`pandas as pd`** for tabular data manipulation (loading CSV, creating DataFrames).  
- **`train_test_split`** and **`LinearRegression`** from **`sklearn`** for splitting data and fitting the baseline regression model.  
- **`r2_score`** and **`mean_absolute_error`** for evaluating model performance.


In [1]:
import os
import joblib
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, cross_val_score

### Load cleaned data

y  → target variable (what we want to predict)

X  → feature matrix 

In [2]:
df = pd.read_csv("../outputs/datasets/collection/HousePricesRecords_clean.csv")
y  = df["Price"]
X  = df.drop(columns=["Price", "Date of Transfer"], errors="ignore")

#### Define numeric & categorical feature lists

Numeric features (continuous variables to be scaled with a `StandardScaler`)  

Categorical features (discrete variables to be one-hot-encoded with a `OneHotEncoder`)

In [3]:
numeric_features = [
    "Year", "Month",
    "RegionMedianPrice", "RegionSaleCount",
    "CountyMedianPrice", "CountySaleCount"
]

categorical_features = [
    "Old/New", "Duration",
    "Town/City", "County", "PPDCategory Type",
    "Property_D", "Property_F", "Property_S", "Property_T",
    "Region"
]

#### Filter Feature Lists to Available Columns

This code ensures that both your numeric and categorical feature lists only include columns that actually exist in the current DataFrame `X`.  
This prevents errors in your preprocessing pipeline if any expected column is missing.

In [4]:
numeric_features     = [c for c in numeric_features     if c in X.columns]
categorical_features = [c for c in categorical_features if c in X.columns]

#### Build Preprocessing & Modeling Pipeline  
This block creates a scikit-learn `ColumnTransformer` named `preprocessor` that:

Scales all numeric features using `StandardScaler` (zero mean, unit variance).  One-hot encodes all categorical features (with unseen categories ignored).

It then defines a `Pipeline` that sequentially:

Applies the `preprocessor` to prepare the data Fits a `RandomForestRegressor` (named “regressor”) on the transformed features.



In [5]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("regressor", RandomForestRegressor(random_state=42))
])

#### Split Data into Training and Test Sets  
This cell uses scikit-learn’s `train_test_split` to randomly split our feature matrix `X` and target vector `y` into:

- **Training set** (`X_train`, `y_train`) comprising 80 % of the data, used to fit the model.  
- **Test set** (`X_test`, `y_test`) comprising 20 % of the data, reserved for evaluating performance on unseen examples.  


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.20,
    random_state=42,  
    shuffle=True
)

#### Hyperparameter Grid Search with Cross-Validation

this cell define a `param_grid` dictionary listing different settings to try for a Random Forest regressor—such as the number of trees, tree depth, and how splits are made. 



In [7]:
param_grid = {
    "regressor__n_estimators":      [100, 200, 300],
    "regressor__max_depth":         [5, 10, 20],
    "regressor__min_samples_split": [2, 5, 10],
    "regressor__min_samples_leaf":  [1, 2, 4],
    "regressor__max_features":      ["auto", "sqrt", "log2"],
    "regressor__bootstrap":         [True, False],
}

then wrap our entire preprocessing-and-model pipeline in a `GridSearchCV` object, asking it to evaluate each combination of these settings using 5-fold cross-validation and to optimise for mean absolute error. When we call `grid_search.fit(X_train, y_train)`, it trains and scores over 2,400 candidate models across different train/validation splits to find the configuration that yields the lowest error.



In [8]:
grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=5,
    scoring="neg_mean_absolute_error",
    n_jobs=-1,
    verbose=1
)

After the search finishes, `grid_search.best_estimator_` holds the pipeline with the optimal hyperparameters. We save that as `best_model` and print out `grid_search.best_params_` so we know exactly which parameter values produced the best results—ready for final evaluation on the test set or for deployment.  

In [9]:
grid_search.fit(X_train, y_train)


best_model = grid_search.best_estimator_
print("Best params:", grid_search.best_params_)

Fitting 5 folds for each of 486 candidates, totalling 2430 fits


/home/cistudent/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
810 fits failed out of a total of 2430.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
429 fits failed with the following error:
Traceback (most recent call last):
  File "/home/cistudent/.local/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/cistudent/.local/lib/python3.12/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/cistudent/.local/lib/python3.12/site-packages/sklearn/pipeline.py", line 4

Best params: {'regressor__bootstrap': True, 'regressor__max_depth': 20, 'regressor__max_features': 'sqrt', 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 10, 'regressor__n_estimators': 100}


#### Check for Any Remaining Non-Numeric Features  
This cell verifies that our feature matrix `X` contains only numeric columns after one-hot encoding. It uses `select_dtypes(exclude=[np.number])` to list any columns that still aren’t numeric. An empty list means you’re safe to proceed; if any names appear, you’ll need to encode or drop those fields before fitting the model.


In [10]:
non_numeric = X.select_dtypes(exclude=[np.number]).columns.tolist()
print("Still non-numeric:", non_numeric)


Still non-numeric: ['Town/City', 'County', 'PPDCategory Type', 'Property_D', 'Property_F', 'Property_S', 'Property_T', 'Region']


#### List Available Training Features  
This cell prints out all column names in `X_train` so you can verify which features are present after splitting. It’s a quick check to ensure your preprocessing and feature lists align with the actual training data.

In [11]:
print("Available columns:", X_train.columns.tolist())

Available columns: ['Old/New', 'Duration', 'Town/City', 'County', 'PPDCategory Type', 'Year', 'Month', 'Property_D', 'Property_F', 'Property_S', 'Property_T', 'Region', 'RegionMedianPrice', 'RegionSaleCount', 'CountyMedianPrice', 'CountySaleCount', 'LogPrice']


#### Evaluate Model Performance on Test Set  
This cell uses the tuned `best_model` to predict sale prices for the held-out `X_test` data, then reports the Mean Absolute Error (MAE) and R² score to show how accurately the model generalises to new, unseen properties.

In [12]:
y_pred = best_model.predict(X_test)
print("📊 Test MAE:", mean_absolute_error(y_test, y_pred))
print("📈 Test R² :", r2_score(y_test, y_pred))



📊 Test MAE: 20858.91501050527
📈 Test R² : 0.4696840577457815


#### Save the Trained Pipeline  
This cell ensures the `outputs/models` folder exists, serialises the tuned `best_model` pipeline to `house_price_pipeline.pkl` using `joblib.dump`, and prints a confirmation so you can load it later for live predictions.



In [13]:
os.makedirs("../outputs/models", exist_ok=True)
joblib.dump(best_model, "../outputs/models/house_price_pipeline.pkl")
print("✅ Pipeline saved to ../outputs/models/house_price_pipeline.pkl")

✅ Pipeline saved to ../outputs/models/house_price_pipeline.pkl
